# 2D Magnetodynamics (Time-Harmonic) - Distribution Transformer

In [1]:
include(joinpath(dirname(@__DIR__), "config.jl"))
paths = get_project_paths("examples")

# Ensure the module is reloaded if changed
if isdefined(Main, :MagnetostaticsFEM)
    println("Reloading MagnetostaticsFEM...")
    # A simple way to force reload in interactive sessions
    try; delete!(LOAD_PATH, paths["SRC_DIR"]); catch; end
    try; delete!(Base.loaded_modules, Base.PkgId(Base.UUID("f8a2b3c4-d5e6-f7a8-b9c0-d1e2f3a4b5c6"), "MagnetostaticsFEM")); catch; end
end
include(joinpath(paths["SRC_DIR"], "MagnetostaticsFEM.jl"))

using LinearAlgebra
using Plots
using LaTeXStrings
using Gmsh: gmsh
using Gridap
using GridapGmsh: GmshDiscreteModel
using .MagnetostaticsFEM
using Printf # For animation title formatting

┌ Warning: Replacing docs for `Main.MagnetostaticsFEM.WeakFormProblem :: Union{}` in module `Main.MagnetostaticsFEM`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.MagnetostaticsFEM.magnetostatics_1d_weak_form :: Tuple{Gridap.Geometry.Triangulation, Gridap.CellData.Measure, AbstractArray, Any, Any}` in module `Main.MagnetostaticsFEM`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.MagnetostaticsFEM.magnetodynamics_1d_harmonic_weak_form :: Tuple{Gridap.Geometry.Triangulation, Gridap.CellData.Measure, AbstractArray, Any, Any, Any, Float64}` in module `Main.MagnetostaticsFEM`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.MagnetostaticsFEM.magnetodynamics_1d_harmonic_coupled_weak_form :: Tuple{Gridap.Geometry.Triangulation, Gridap.CellData.Measure, AbstractArray, Any, Any, Any, Float64}` in module `Main.MagnetostaticsFEM`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.MagnetostaticsFEM.magnetodynamics_2d_harmon

## Define Parameters and Paths

In [2]:
# Model Parameters
μ0 = 4e-7 * pi  # Vacuum permeability [H/m]
μr_core = 1000.0 # Relative permeability of the core (linear case)
σ_core = 0.1    # Conductivity of the core [S/m] (Laminated)
freq = 50.0     # Frequency [Hz]
ω = 2 * pi * freq # Angular frequency [rad/s]

# Source Parameters (Secondary Current Driven, Primary Open)
Ip = 0.0       # Primary peak phase current [A]
Is = 777.62    # Secondary peak phase current [A]
Np = 266       # Number of primary turns
Ns = 6         # Number of secondary turns

# Winding Areas (Approximate - needed for current density)
# These values seem missing in the original notebook, using placeholders.
# Need to calculate these from the geometry definition if possible, or use values from source.
Awhv_approx = 6000e-6 # Approximate area HV winding region [m^2]
Awlv_approx = 3000e-6 # Approximate area LV winding region [m^2]

Jp = Np * Ip / Awhv_approx # Peak current density primary [A/m^2]
Js = Ns * Is / Awlv_approx # Peak current density secondary [A/m^2]

# FEM Parameters
order = 1 # Linear elements match original notebook
field_type = ComplexF64 # Use ComplexF64 marker for setup_fe_spaces
dirichlet_tag_name = "Enclosure" # Name of the physical group for Dirichlet BC
dirichlet_value = 0.0 + 0.0im # Dirichlet BC for Az = u + iv

# Paths
mesh_file = joinpath(paths["GEO_DIR"], "transformer_stedin.msh")
output_file_base = joinpath(paths["OUTPUT_DIR"], "transformer_linear_dynamics")

println("Mesh file: ", mesh_file)
println("Output directory: ", paths["OUTPUT_DIR"])

Mesh file: /Users/ezracerpac/PycharmProjects/FutureDistributionSystemsAM/examples/geo/transformer_stedin.msh
Output directory: /Users/ezracerpac/PycharmProjects/FutureDistributionSystemsAM/examples/output


## Setup FEM Problem (Linear Magnetodynamics)

In [5]:
# Load mesh and tags
model, labels, tags = load_mesh_and_tags(mesh_file)

# Get material tags dictionary using the 2D specific function
material_tags = get_material_tags_2d(labels)
println("Material Tags: ", material_tags)

# Set up triangulation and measures
Ω = Triangulation(model)
dΩ = Measure(Ω, 2*order)

# Define material property functions
# Use 'Oil' as the default low permeability region if 'Air' is not present
air_tag_name = haskey(material_tags, "Air") ? "Air" : "Oil"
reluctivity_func = define_reluctivity(material_tags, μ0, μr_core; core_tag_name="Core", air_tag_name=air_tag_name)
conductivity_func = define_conductivity(material_tags, σ_core; core_tag_name="Core")

# Define source current density function (complex phasors)
function source_current_density_2d(tags_dict, Jp, Js)
    # Use get with default to avoid errors if a tag is missing
    tag_hv1_l = get(tags_dict, "HV1l", -1)
    tag_hv1_r = get(tags_dict, "HV1r", -1)
    tag_hv2_l = get(tags_dict, "HV2l", -1)
    tag_hv2_r = get(tags_dict, "HV2r", -1)
    tag_hv3_l = get(tags_dict, "HV3l", -1)
    tag_hv3_r = get(tags_dict, "HV3r", -1)
    tag_lv1_l = get(tags_dict, "LV1l", -1)
    tag_lv1_r = get(tags_dict, "LV1r", -1)
    tag_lv2_l = get(tags_dict, "LV2l", -1)
    tag_lv2_r = get(tags_dict, "LV2r", -1)
    tag_lv3_l = get(tags_dict, "LV3l", -1)
    tag_lv3_r = get(tags_dict, "LV3r", -1)
    
    phase1 = exp(1im * 2pi/3)
    phase2 = 1.0 + 0.0im
    phase3 = exp(-1im * 2pi/3)
    
    function Jz(tag)
        current = 0.0 + 0.0im
        # Primary (HV)
        if tag == tag_hv1_l; current -= Jp * phase1; end
        if tag == tag_hv1_r; current += Jp * phase1; end
        if tag == tag_hv2_l; current -= Jp * phase2; end
        if tag == tag_hv2_r; current += Jp * phase2; end
        if tag == tag_hv3_l; current -= Jp * phase3; end
        if tag == tag_hv3_r; current += Jp * phase3; end
        # Secondary (LV)
        if tag == tag_lv1_l; current += Js * phase1; end
        if tag == tag_lv1_r; current -= Js * phase1; end
        if tag == tag_lv2_l; current += Js * phase2; end
        if tag == tag_lv2_r; current -= Js * phase2; end
        if tag == tag_lv3_l; current += Js * phase3; end
        if tag == tag_lv3_r; current -= Js * phase3; end
        return current
    end
    return Jz
end

# Define the actual complex source current density function
source_current_complex_func = source_current_density_2d(material_tags, Jp, Js)

# For the coupled real/imaginary weak form, we still need a *real* reference source.
# Let's use the magnitude of the phase 2 secondary current density.
J0_real_ref = Js # Use magnitude of secondary current density as reference
# Define the positive/negative tags based on the actual mesh physical names
source_current_func_real = define_current_density(material_tags, J0_real_ref; 
                                            coil_tags_pos=["LV2l"], 
                                            coil_tags_neg=["LV2r"])
# TODO: The coupled weak form using a real source is an approximation.
# A full complex source requires a complex weak form or modification of the coupled form.

# Setup FE spaces (multi-field: Real, Imag parts)
dirichlet_tag_id = get_tag_from_name(labels, dirichlet_tag_name)
U, V = setup_fe_spaces(model, order, field_type, dirichlet_tag_name, dirichlet_value)

# Define the weak form problem for the coupled system using the real reference source
problem = magnetodynamics_1d_harmonic_coupled_weak_form(Ω, dΩ, tags, reluctivity_func, conductivity_func, source_current_func_real, ω)

Info    : Reading '/Users/ezracerpac/PycharmProjects/FutureDistributionSystemsAM/examples/geo/transformer_stedin.msh'...
Info    : 168 entities
Info    : 10393 nodes
Info    : 20784 elements
Info    : Done reading '/Users/ezracerpac/PycharmProjects/FutureDistributionSystemsAM/examples/geo/transformer_stedin.msh'
Material Tags: Dict("Core" => 3, "HV1l" => 4, "HV2l" => 6, "HV3r" => 9, "LV3r" => 15, "LV windings" => 17, "LV3l" => 14, "Air" => 2, "HV3l" => 8, "HV1r" => 5, "LV1l" => 10, "HV windings" => 16, "Oil" => 2, "LV2r" => 13, "LV1r" => 11, "LV2l" => 12, "HV2r" => 7)
Setting up multi-field spaces (Real, Imag) for Complex problem.


WeakFormProblem(Main.MagnetostaticsFEM.var"#a#45"{Gridap.CellData.GenericMeasure, Float64, Gridap.CellData.OperationCellField{ReferenceDomain}, Gridap.CellData.OperationCellField{ReferenceDomain}}(GenericMeasure(), 314.1592653589793, OperationCellField(), OperationCellField()), Main.MagnetostaticsFEM.var"#b#46"{Gridap.CellData.GenericMeasure, Gridap.CellData.OperationCellField{ReferenceDomain}}(GenericMeasure(), OperationCellField()))

## Solve FEM Problem

In [6]:
# Solve the real coupled linear FE system
uv = solve_fem_problem(problem, U, V) # uv is a MultiFieldFEFunction

# Extract real and imaginary parts
u = uv[1] # Real part of Az
v = uv[2] # Imag part of Az

SingleFieldFEFunction():
 num_cells: 20560
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 10068253004045308322

## Post-processing

In [7]:
# Compute B-field (Real and Imag parts)
B_re, B_im = calculate_b_field(uv)

# Compute Eddy Currents (Real and Imag parts)
J_eddy_re, J_eddy_im = calculate_eddy_current(uv, conductivity_func, ω, Ω, tags)

# Define helper functions for magnitude squared
mag_sq_scalar(re, im) = re*re + im*im
mag_sq_vector(re, im) = inner(re, re) + inner(im, im)

# Calculate Magnitudes for saving/plotting using composition
Az_mag = sqrt ∘ (mag_sq_scalar ∘ (u, v))
B_mag = sqrt ∘ (mag_sq_vector ∘ (B_re, B_im))
Jeddy_mag = sqrt ∘ (mag_sq_scalar ∘ (J_eddy_re, J_eddy_im))

# Calculate total current density magnitude (approximation)
# J_total_re = source_current_func_real(tags) + J_eddy_re # Need CellField source
# J_total_im = J_eddy_im
# J_total_mag = sqrt ∘ (mag_sq_scalar ∘ (J_total_re, J_total_im))
# TODO: Need a better way to handle source current in post-processing

# Save results to VTK format
save_results_vtk(Ω, output_file_base, 
    Dict(
        "Az_re" => u, "Az_im" => v, "Az_mag" => Az_mag,
        "B_re" => B_re, "B_im" => B_im, "B_mag" => B_mag,
        "Jeddy_re" => J_eddy_re, "Jeddy_im" => J_eddy_im, "Jeddy_mag" => Jeddy_mag
        # "Jtotal_mag" => J_total_mag
    ))

Results saved to /Users/ezracerpac/PycharmProjects/FutureDistributionSystemsAM/examples/output/transformer_linear_dynamics.vtu


## Next Steps

1.  **Refine Source Term:** Implement the full complex source current density, potentially requiring a modification to the weak form or using a direct complex formulation if `Gridap` supports it easily.
2.  **Non-Linearity:** Extend `MagnetostaticsFEM.jl` to handle non-linear reluctivity (BH curve) using an iterative solver.
3.  **Voltage-Driven Coils:** Extend `MagnetostaticsFEM.jl` to handle coupled circuit equations using `MultiFieldFESpace`.